# Geolocation features and Cluster model
Once we already know how the stations behave each hour, on weekdays, we will add geolocation features to our model. Do they behave the same depending on how high in the city the stations are located? Is there any relationship between how close to the city centre they are? We will find this out in this notebook.

In [1]:
#Importing required libraries
import pandas as pd
import geopy.distance
import geocoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
#importing datetime_features file generated from 1_datetime
oct_features = pd.read_csv("..\\Dataset\\datetime_features.csv",encoding="utf_8",index_col='station_id')
#importing dataframe extracted from web scrapping Bicing website, thanks to Laurent Guerguy https://github.com/laurent-guerguy
coordinates_df = pd.read_csv("..\\Dataset\\bicing_ws_laurent.csv",encoding="utf_8")

In [3]:
#We only need 'station_id', latitude and longitude columns
coordinates_df = coordinates_df[['1','4','5']]

In [4]:
#We double check column '1' refers to station_id
coordinates_df['1'].unique()
print(f"The dataframe has the stations from its id number {coordinates_df['1'].min()} to the {coordinates_df['1'].max()}")

The dataframe has the stations from its id number 1 to the 496


In [5]:
coordinates_df = coordinates_df.rename(columns={"1":'station_id',"4":"lat","5":"long"})

In [6]:
coordinates_df = coordinates_df.groupby('station_id').mean()

In [7]:
oct_features_gps = pd.merge(oct_features, coordinates_df, on="station_id")

In [8]:
#Checking if we are missing something
oct_features_gps.isnull().sum()

00:00       2
01:00       2
02:00       2
03:00       2
04:00       2
05:00       2
06:00       2
07:00       2
08:00       2
09:00       2
10:00       2
11:00       2
12:00       2
13:00       2
14:00       2
15:00       2
16:00       2
17:00       2
18:00       2
19:00       2
20:00       2
21:00       2
22:00       2
23:00       2
capacity    0
lat         0
long        0
dtype: int64

In [9]:
oct_features_gps[pd.isnull(oct_features_gps).any(axis=1)]

,00:00,01:00,02:00,03:00,04:00,05:00,06:00,07:00,08:00,09:00,...,17:00,18:00,19:00,20:00,21:00,22:00,23:00,capacity,lat,long
station_id,,,,,,,,,,,,,,,,,,,,,
311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,41.378638,2.120393
425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,41.376520,2.174900


In [10]:
#We drop these two stations as their maximum capacity is 0, which means they were not operative
oct_features_gps.dropna(inplace=True)

### Calculating the distance of each station to the city centre
We will use Plaça Catalunya, Lat: 41.3870154, Long: 2.1700471 as the centre of the city and will caluclate the distance, in kilometres, of the station to the centre.

In [11]:
PlCat = (41.3870154, 2.170047)

In [12]:
#Iterating between stations to find the distance from Plaça Ctalunya
stations = range(oct_features_gps.shape[0])
distances = []
for s in stations:
    location = (oct_features_gps.iloc[s]['lat'],oct_features_gps.iloc[s]['long'])
    distances.append(round(geopy.distance.vincenty(PlCat, location).km,2))

C:\Users\jaume\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


In [13]:
#Adding distances as feature
oct_features_gps['DisttoCentre'] = distances

In [14]:
oct_features_gps.head()

,00:00,01:00,02:00,03:00,04:00,05:00,06:00,07:00,08:00,09:00,...,18:00,19:00,20:00,21:00,22:00,23:00,capacity,lat,long,DisttoCentre
station_id,,,,,,,,,,,,,,,,,,,,,
1,78.0,79.0,79.0,80.0,79.0,70.0,39.0,13.0,17.0,20.0,...,40.0,42.0,53.0,59.0,67.0,73.0,30,41.397952,2.180042,1.47
2,50.0,52.0,51.0,50.0,50.0,44.0,27.0,13.0,12.0,16.0,...,26.0,27.0,33.0,38.0,46.0,47.0,27,41.395530,2.177060,1.11
3,61.0,62.0,62.0,62.0,61.0,62.0,53.0,29.0,15.0,13.0,...,72.0,68.0,61.0,60.0,61.0,64.0,27,41.394055,2.181299,1.22
4,47.0,46.0,47.0,49.0,55.0,67.0,48.0,23.0,10.0,13.0,...,74.0,64.0,55.0,46.0,46.0,50.0,21,41.393480,2.181555,1.20
5,63.0,63.0,62.0,61.0,62.0,59.0,48.0,24.0,21.0,26.0,...,78.0,69.0,69.0,70.0,69.0,64.0,39,41.391075,2.180223,0.96


# Building the model 1
Now that we have all features ready to go, we will build the model based on those to see which is the output. Initially will work with 5 clusters, expecting, hypothetically, the following:
* One cluster for residential stations
* One cluster for office stations
* One cluster for stations located in elevated points of the city
* One cluster for very central/hybrid stations (that have bikes in and out constantly)
* One cluster for seaside stations

In [15]:
kmeans = KMeans(n_clusters=5)
october_clusters = kmeans.fit(oct_features_gps)
october_clusters.cluster_centers_

array([[48.50617284, 53.43209877, 57.50617284, 60.01234568, 59.09876543,
        45.90123457, 25.32098765, 14.69135802, 13.04938272, 13.25925926,
        13.90123457, 15.0617284 , 15.0617284 , 14.64197531, 14.33333333,
        14.77777778, 16.34567901, 17.81481481, 20.58024691, 25.07407407,
        29.44444444, 31.28395062, 34.50617284, 40.33333333, 24.19753086,
        41.40395617,  2.15988784,  3.10382716],
       [11.87128713, 13.92079208, 16.11881188, 19.16831683, 20.89108911,
        19.02970297, 15.26732673, 13.91089109, 15.12871287, 15.0990099 ,
        14.61386139, 13.95049505, 10.68316832,  8.24752475,  8.75247525,
         9.51485149,  9.32673267, 10.18811881, 11.36633663, 11.79207921,
        12.01980198, 11.17821782, 10.03960396, 10.17821782, 23.53465347,
        41.39663666,  2.15195311,  2.74554455],
       [54.425     , 50.25      , 45.4375    , 41.0875    , 38.425     ,
        37.0875    , 42.5125    , 56.5625    , 66.4       , 69.3125    ,
        69.1125    , 67.5   

# Building the model 2: With PCA
As we are having more than 25 features, it might make sense to reduce the number of components into 5 and compare with the result of the model 1.

In [16]:
# Principal Component 
pca = PCA(n_components=5)

principalComponents = pca.fit_transform(oct_features_gps)
principalDf = pd.DataFrame(data = principalComponents
             ,columns = ['pc1', 'pc2', 'pc3', 'pc4','pc5'])
principalDf.head()    

,pc1,pc2,pc3,pc4,pc5
0,68.260210,-71.154738,33.906180,12.814888,-11.348020
1,-1.201679,-48.749910,-3.832702,-5.845796,-17.750891
2,90.174332,-39.994895,-12.292043,-38.994167,6.103862
3,67.101562,-21.241376,-25.217160,-60.311126,-5.377992
4,84.400931,-44.989076,-9.342367,-16.037285,29.467895


In [17]:
kmeans = KMeans(n_clusters=5)
october_clusters_PCA = kmeans.fit(principalDf)
october_clusters_PCA.cluster_centers_

array([[-117.15704644,  -28.62383986,  -20.69041916,   -5.6203072 ,
           1.98639096],
       [ -30.87262028,  -69.87953521,   22.87804602,    6.79918023,
          -5.35826746],
       [ -55.41688403,  116.20004047,   20.25551077,    3.5078357 ,
           0.89128504],
       [  90.96946008,   69.22414434,  -16.93862922,    3.60203591,
          -1.21723865],
       [ 131.58129121,  -56.04548148,    3.39620673,   -6.16363906,
           3.36839188]])

# Adding the models to our dataframe

In [18]:
oct_features_gps['labels'] = october_clusters.fit_predict(oct_features_gps)
oct_features_gps.head()

,00:00,01:00,02:00,03:00,04:00,05:00,06:00,07:00,08:00,09:00,...,19:00,20:00,21:00,22:00,23:00,capacity,lat,long,DisttoCentre,labels
station_id,,,,,,,,,,,,,,,,,,,,,
1,78.0,79.0,79.0,80.0,79.0,70.0,39.0,13.0,17.0,20.0,...,42.0,53.0,59.0,67.0,73.0,30,41.397952,2.180042,1.47,3
2,50.0,52.0,51.0,50.0,50.0,44.0,27.0,13.0,12.0,16.0,...,27.0,33.0,38.0,46.0,47.0,27,41.395530,2.177060,1.11,4
3,61.0,62.0,62.0,62.0,61.0,62.0,53.0,29.0,15.0,13.0,...,68.0,61.0,60.0,61.0,64.0,27,41.394055,2.181299,1.22,3
4,47.0,46.0,47.0,49.0,55.0,67.0,48.0,23.0,10.0,13.0,...,64.0,55.0,46.0,46.0,50.0,21,41.393480,2.181555,1.20,3
5,63.0,63.0,62.0,61.0,62.0,59.0,48.0,24.0,21.0,26.0,...,69.0,69.0,70.0,69.0,64.0,39,41.391075,2.180223,0.96,3


In [19]:
oct_features_gps['labelsPCA'] = october_clusters_PCA.fit_predict(principalDf)
oct_features_gps.head()

,00:00,01:00,02:00,03:00,04:00,05:00,06:00,07:00,08:00,09:00,...,20:00,21:00,22:00,23:00,capacity,lat,long,DisttoCentre,labels,labelsPCA
station_id,,,,,,,,,,,,,,,,,,,,,
1,78.0,79.0,79.0,80.0,79.0,70.0,39.0,13.0,17.0,20.0,...,53.0,59.0,67.0,73.0,30,41.397952,2.180042,1.47,3,3
2,50.0,52.0,51.0,50.0,50.0,44.0,27.0,13.0,12.0,16.0,...,33.0,38.0,46.0,47.0,27,41.395530,2.177060,1.11,4,2
3,61.0,62.0,62.0,62.0,61.0,62.0,53.0,29.0,15.0,13.0,...,61.0,60.0,61.0,64.0,27,41.394055,2.181299,1.22,3,3
4,47.0,46.0,47.0,49.0,55.0,67.0,48.0,23.0,10.0,13.0,...,55.0,46.0,46.0,50.0,21,41.393480,2.181555,1.20,3,3
5,63.0,63.0,62.0,61.0,62.0,59.0,48.0,24.0,21.0,26.0,...,69.0,70.0,69.0,64.0,39,41.391075,2.180223,0.96,3,3


In [26]:
oct_features_gps[['labels','labelsPCA']] = oct_features_gps[['labels','labelsPCA']].replace([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

In [27]:
oct_features_gps.labels.value_counts()

2    101
5     81
4     80
1     79
3     67
Name: labels, dtype: int64

In [28]:
oct_features_gps.labelsPCA.value_counts()

5    101
3     81
1     81
4     80
2     65
Name: labelsPCA, dtype: int64

In [29]:
#Let's check if there is any relationship between the labels created by the two models:
oct_features_gps.groupby(["labels","labelsPCA"]).count()

00:00  01:00  02:00  03:00  04:00  05:00  06:00  07:00  \
labels labelsPCA                                                           
1      1             79     79     79     79     79     79     79     79   
2      5            101    101    101    101    101    101    101    101   
3      1              2      2      2      2      2      2      2      2   
       2             65     65     65     65     65     65     65     65   
4      4             80     80     80     80     80     80     80     80   
5      3             81     81     81     81     81     81     81     81   

                  08:00  09:00  ...  18:00  19:00  20:00  21:00  22:00  23:00  \
labels labelsPCA                ...                                             
1      1             79     79  ...     79     79     79     79     79     79   
2      5            101    101  ...    101    101    101    101    101    101   
3      1              2      2  ...      2      2      2      2      2      2   
       2             65     65  ...     65     65     65     65     65     65   
4      4             80     80  ...     80     80     80     80     80     80   
5      3             81     81  ...     81     81     81     81     81     81   

                  capacity  lat  long  DisttoCentre  
labels labelsPCA                                     
1      1                79   79    79            79  
2      5               101  101   101           101  
3      1                 2    2     2             2  
       2                65   65    65            65  
4      4                80   80    80            80  
5      3                81   81    81            81  

[6 rows x 28 columns]

In [31]:
print(f'Out of {oct_features_gps.shape[0]} stations with its labels, only 2 of them have labels that do not match together.')
#Although the models have numbered the labels with different numbers we consider them equivalent

Out of 408 stations with its labels, only 2 of them have labels that do not match together.


For this reason, we will focus our analysis in only the 'Label' classification done by the Model 1.

In [32]:
oct_features_gps.to_csv("..\\Dataset\\clusters.csv",encoding="utf_8",decimal=',', sep=';', index=True)